# Compile YoloV5 model for AWS neuron

* AWS inf1 use aws neuron sdk [link](https://github.com/aws/aws-neuron-sdk)
* AWS Sample code for inf1 pytorch [link](https://github.com/aws/aws-neuron-sdk/blob/master/docs/pytorch-neuron/tutorial-compile-infer.md)
* neuron installation guide [link](https://github.com/aws/aws-neuron-sdk/blob/master/docs/neuron-install-guide.md)
* can't compile model. file an [issue](https://github.com/aws/aws-neuron-sdk/issues/141)

In [ ]:
!pip install tensorboard onnx torch torchvision --upgrade

In [ ]:
!pip install neuron-cc torch-neuron --extra-index-url="https://pip.repos.neuron.amazonaws.com"

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [1]:
import sys, os, onnx, torch
import torch_neuron

fp_yolov5 = os.path.abspath(os.getcwd()) + "/yolov5"
sys.path.append(fp_yolov5)
sys.path

['',
 '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python36.zip',
 '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6',
 '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/lib-dynload',
 '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages',
 '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython',
 '/home/ec2-user/SageMaker/myAWSStudyBlog/yolov5-onnx-sm/6-yolov5-inf1/yolov5']

In [2]:
%cd ./yolov5

/home/ec2-user/SageMaker/myAWSStudyBlog/yolov5-onnx-sm/6-yolov5-inf1/yolov5


In [3]:
!python -c "from utils.google_utils import *; attempt_download('weights/yolov5s.pt');"

In [8]:
img = torch.zeros((1, 3, 640, 640))
fp_model = './weights/yolov5s.pt'
model = torch.load(fp_model, map_location=torch.device('cpu'))['model'].float()
model.eval()
model.model[-1].export = False
y = model(img)  # dry run to inference mode

model_neuron = torch.neuron.trace(model, example_inputs=[img])
model_neuron.save("yolov5_neuron.pt")

/home/ec2-user/SageMaker/myAWSStudyBlog/yolov5-onnx-sm/6-yolov5-inf1/yolov5/models/yolo.py:32: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.grid[i].shape[2:4] != x[i].shape[2:4]:


TypeError: copy_() missing 1 required positional arguments: "other"

In [ ]:
torch.__version__

In [5]:
!cp ../../2-yolov5-onnx/yolov5s_detect.onnx yolov5s_detect.onnx

In [7]:
!neuron-cc compile --framework ONNX yolov5s_detect.onnx \
--output ./onnx.neff --io-config '{"inputs":{"images":[[1,3,640,640], "float32"]},"output":[[1,25200,85], "float32"]}'

07/28/2020 08:25:51 AM ERROR [neuron-cc]: ***************************************************************
07/28/2020 08:25:51 AM ERROR [neuron-cc]:  An Internal Compiler Error has occurred
07/28/2020 08:25:51 AM ERROR [neuron-cc]: ***************************************************************
07/28/2020 08:25:51 AM ERROR [neuron-cc]: 
07/28/2020 08:25:51 AM ERROR [neuron-cc]: Error message:  A process in the process pool was terminated abruptly while the future was running or pending.
07/28/2020 08:25:51 AM ERROR [neuron-cc]: 
07/28/2020 08:25:51 AM ERROR [neuron-cc]: Error class:    BrokenProcessPool
07/28/2020 08:25:51 AM ERROR [neuron-cc]: Error location: pipeline.compile.0
07/28/2020 08:25:51 AM ERROR [neuron-cc]: Command line:   /home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/neuron-cc compile --framework ONNX yolov5s_detect.onnx --output ./onnx.neff --io-config '{"inputs":{"images":[[1,3,640,640], "float32"]},"output":[[1,25200,85], "float32"]}'
07/28/2020 08:25:51 AM ERROR

In [ ]:
neuron-cc compile \
  --framework ONNX resnet50/model.onnx \
  --output onnx.neff \
  --io-config '{"inputs":{"gpu_0/data_0":[[1,3,224,224], "float32"]},"outputs":["gpu_0/softmax_1"]}'